Import Libraries :

In [543]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

DataSet :

In [544]:
data_fake = pd.read_csv("C:\\Users\\vishn\\Downloads\\Fake.csv")
data_true = pd.read_csv("C:\\Users\\vishn\\Downloads\\True.csv")


In [545]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [546]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Data Preparation :

In [547]:
data_fake['class'] = 0
data_true['class'] = 1

In [548]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [549]:
data_fake_manual = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i], axis = 0 , inplace = True)
    

In [550]:
data_true_manual = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i], axis = 0 , inplace = True) 

In [551]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [552]:
df_manual_testing = pd.concat([data_fake_manual, data_true_manual], axis=0)
df_manual_testing.to_csv("C:\\Users\\vishn\\Downloads\\Fake_news\\Fake_news\\manual_testing.csv")

In [588]:
data_merge = pd.concat([data_fake_manual, data_true_manual], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [554]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [555]:
data=data_merge.drop(['title','subject','date'],axis=1)
data.head(10)

,text,class
23471,"21st Century Wire says This week, the historic...",0
23472,By Dady Chery and Gilbert MercierAll writers ...,0
23473,Vic Bishop Waking TimesOur reality is carefull...,0
23474,Paul Craig RobertsIn the last years of the 20t...,0
23475,Robert Fantina CounterpunchAlthough the United...,0
23476,21st Century Wire says As 21WIRE reported earl...,0
23477,21st Century Wire says It s a familiar theme. ...,0
23478,Patrick Henningsen 21st Century WireRemember ...,0
23479,21st Century Wire says Al Jazeera America will...,0
23480,21st Century Wire says As 21WIRE predicted in ...,0


In [556]:
data = data.sample(frac=1)

In [557]:
data.head()


,text,class
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,1
23476,21st Century Wire says As 21WIRE reported earl...,0
23474,Paul Craig RobertsIn the last years of the 20t...,0
23472,By Dady Chery and Gilbert MercierAll writers ...,0
23471,"21st Century Wire says This week, the historic...",0


In [558]:
data.isnull().sum()

text     0
class    0
dtype: int64

Data Preprocessing : 

In [559]:
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+','', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [560]:
data['text']=data['text'].apply(word_drop)

In [561]:
data.head(10)

,text,class
21415,moscow reuters vatican secretary of state ...,1
23476,century wire says as reported earlier this w...,0
23474,paul craig robertsin the last years of the ce...,0
23472,by dady chery and gilbert mercierall writers ...,0
23471,century wire says this week the historic int...,0
23478,patrick henningsen century wireremember when...,0
23473,vic bishop waking timesour reality is carefull...,0
23475,robert fantina counterpunchalthough the united...,0
23477,century wire says it s a familiar theme when...,0
21407,sao paulo reuters cesar mata pires the ow...,1


Feature and Label Extraction:

In [562]:
x = data['text']
y = data['class']

Train-Test Split:

In [563]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

Vectorization :

In [564]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

Model Training and Evaluation:

In [565]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [566]:
LR.score(xv_test, y_test)

0.8

In [567]:
pre_lr = LR.predict(xv_test)

In [568]:
print(classification_report(y_test, pre_lr))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5



In [569]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [570]:
DT.score(xv_test, y_test)

1.0

In [571]:
pred_dt = DT.predict(xv_test)

In [572]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [573]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state = 0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [574]:
GBC.score(xv_test, y_test)

1.0

In [575]:
pred_gbc = GBC.predict(xv_test)

In [576]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [577]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state = 0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [578]:
RFC.score(xv_test, y_test)

1.0

In [579]:
pred_rfc = RFC.predict(xv_test)

In [580]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



Prediction Function :

In [590]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"


Manual_Testing Function :

In [583]:
def manual_testing(news):
    try:
        # Step 1: Create DataFrame for the input news
        testing_news = {"text": [news]}
        new_def_test = pd.DataFrame(testing_news)

        # Step 2: Preprocess the text using wordopt function
        new_def_test["text"] = new_def_test["text"].apply(wordopt)
        print("Preprocessed text:", new_def_test["text"].iloc[0])  # Debugging line to check preprocessing

        # Step 3: Transform the preprocessed text into vectorized form
        new_xv_test = vectorization.transform(new_def_test["text"])
        print("Vectorized text shape:", new_xv_test.shape)  # Debugging line to check vectorization

        # Step 4: Perform predictions using different models
        pre_LR = LR.predict(new_xv_test)
        pred_DT = DT.predict(new_xv_test)
        pred_GBC = GBC.predict(new_xv_test)
        pred_RFC = RFC.predict(new_xv_test)

        # Step 5: Output the predictions
        return print("\n\nLR prediction: {} \nDT prediction: {} \nGBC prediction: {} \nRFC prediction: {}"
                     .format(output_label(pre_LR[0]),
                             output_label(pred_DT[0]),
                             output_label(pred_GBC[0]),
                             output_label(pred_RFC[0])))
    except Exception as e:
        print("Error during manual testing:", e)

In [587]:
news = str(input())
manual_testing(news)

 Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, 

Preprocessed text: vic bishop waking timesour reality is carefully constructed by powerful corporate  political and special interest sources in order to covertly sway public opinion  blatant lies are often televised regarding terrorism  food  war  health  etc  they are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms the practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this  the entire role of this brainwashing industry is to figure out how to spin information to journalists  similar to the lobbying of government  it is never really clear just how much truth the journalists receive because the news industry has become complacent  the messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own   of the media general electric  ge   news corp  disney  viaco

In [586]:
news = str(input())
manual_testing(news)

 MINSK (Reuters) - In the shadow of disused Soviet-era factories in Minsk, a street lined with eclectic bars, art galleries and yoga studios has become a haven from the vigilant eyes of the Belarussian authorities.  This place is like an island,  said Yegor, 21, who works at  popular bar Hooligan.  It s the street of freedom.  The government of President Alexander Lukashenko, who has ruled Belarus for the past 23 years and has boasted that he is  the last and only dictator in Europe  ,has little tolerance for any opposition. A powerful police force and feared state security  keep citizens in check. But police patrols are rare in Oktyabrskaya, partly due to its location on an out-of-the-way peninsula in a bend of the river Svislach. When the first restaurant opened there in 2012, few visitors came but now it ranks among the most fashionable quarters of Minsk.  Such is the growing popularity of that Oktyabrskaya that investors such as Belgazprombank, a subsidiary of state-owned Russian l

Preprocessed text: minsk  reuters    in the shadow of disused soviet era factories in minsk  a street lined with eclectic bars  art galleries and yoga studios has become a haven from the vigilant eyes of the belarussian authorities   this place is like an island   said yegor    who works at  popular bar hooligan   it s the street of freedom   the government of president alexander lukashenko  who has ruled belarus for the past  years and has boasted that he is  the last and only dictator in europe   has little tolerance for any opposition  a powerful police force and feared state security  keep citizens in check  but police patrols are rare in oktyabrskaya  partly due to its location on an out of the way peninsula in a bend of the river svislach  when the first restaurant opened there in   few visitors came but now it ranks among the most fashionable quarters of minsk   such is the growing popularity of that oktyabrskaya that investors such as belgazprombank  a subsidiary of state owned